<a href="https://colab.research.google.com/github/arthurst38/deep_learning/blob/main/Reconnaissance_d'auteur_avec_des_RNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reconnaissance d'auteur avec des RNNs

Dans ce TP, nous allons utiliser des RNNs pour retrouver l'auteur d'un texte.

Le corpus utilisé est tiré d'œuvres littéraires classiques de littérature anglophone : 9 auteurs, 2 livres par auteurs avec un fichier par chapitre.

## Téléchargement des données depuis un répo git

In [ ]:
!git clone https://github.com/nzmonzmp/dataset-9classical-author.git

## Téléchargement du modèle spacy pour l'anglais

In [ ]:
!python -m spacy download en

## Import de TensorFlow et des autres librairies nécessaires

In [ ]:
import re
import typing

import numpy
import pathlib
import seaborn
import sklearn.model_selection
import sklearn.preprocessing
import spacy
import tensorflow.keras as keras
import tqdm.notebook

## Prétraitements

- Chaque phrase est un exemple
- Sont remplacés par leur type tout mot (ou groupe de mots) étant une entité nommée de type :
  - Personne
  - Bâtiment
  - Institution
  - Lieu
  - Date précise
  - Monnaie
  - Œuvre artistique

In [ ]:
nlp = spacy.load('en_core_web_sm')
ner_to_replace= dict(PERSON="person",
                     FAC="building" , 
                     ORG="organization", 
                     GPE="city" ,
                     LOC="lake" , 
                     DATE="date" , 
                     MONEY="dollar" , 
                     WORK_OF_ART="painting")


def get_data(directory: pathlib.Path
            ) -> typing.Tuple[typing.List[str], typing.List[str]]:
  texts = []
  authors = []
  for item in tqdm.notebook.tqdm(list(directory.glob("*/*/*.txt"))):
    author = item.parent.parent.name
    text = item.read_text(encoding="utf8")
    doc = nlp(" ".join(text.split()))
    for s in doc.sents:
      seq = [(ner_to_replace[t.ent_type_]
              if t.ent_type_ in ner_to_replace
              else t.text)
             for t in s
             if t.ent_iob_ != "I"]
      if not re.search("chapter", seq[0], re.IGNORECASE):
        texts.append((" ".join(seq)))
        authors.append(author)
  return texts, authors

texts, authors = get_data(pathlib.Path("dataset-9classical-author"))
print(len(texts), len(authors))


label_encoder = sklearn.preprocessing.LabelEncoder()
y = label_encoder.fit_transform(authors)
print(y)
X_train_raw, X_test_raw, y_train, y_test = \
    sklearn.model_selection.train_test_split(texts, y, test_size=0.3)

In [ ]:
print(len(X_train_raw), y_train.shape)

## Préparation des données en séquences



### Préparation du dictionnaire et des séquences

À l'aide de l'outil [`tensorflow.keras.preprocessing.text.Tokenizer`](https://keras.io/api/preprocessing/text/#tokenizer-class) :
- Constituez un dictionnaire sur le Corpus `X_train_raw`
- Quelle est la taille du vocabulaire ?
- Quelle est la taille maximum, en nombre de mots, d'une phrase ?

In [ ]:
# Votre code ici

#### Solution

In [ ]:
tokenizer_obj = keras.preprocessing.text.Tokenizer()
tokenizer_obj.fit_on_texts(X_train_raw)

In [ ]:
vocab_size = len(tokenizer_obj.word_index) + 1
max_length = max(len(s.split()) for s in X_train_raw)
print(f"Taille du vocabulaire : {vocab_size}")
print(f"Taille de la plus grande séquence de mot : {max_length}")

### Création de la matrice de séquences d'indices

À l'aide des outils [`tensorflow.keras.preprocessing.text.Tokenizer`](https://keras.io/api/preprocessing/text/#tokenizer-class) et [`tensorflow.keras.preprocessing.sequence.pad_sequences`](https://keras.io/api/preprocessing/timeseries/#padsequences-function) :
- Transformez `X_train_raw` et `X_test_raw` en séquences d'indices de mots dans un dictionnaire. (fonction ``texts_to_sequences()`` de l'objet ``Tokenizer`` instancié précédemment)
- Effectuez l'opération de **padding** sur les séquences afin qu'elles aient une taille raisonnable pour être traitées par un GRU bidirectionnel :
  - On utilisera `maxlen = 150` car au-delà les GRU commencent à ne plus être capable de transmettre correctement l'information.
- Stockez les séquences obtenues dans `X_train_pad` et `X_test_pad`

In [ ]:
# Votre code ici

#### Solution

In [ ]:
max_length = 150

X_train_tokens = tokenizer_obj.texts_to_sequences(X_train_raw)
X_test_tokens = tokenizer_obj.texts_to_sequences(X_test_raw)

X_train_pad = keras.preprocessing.sequence.pad_sequences(
    X_train_tokens, maxlen=max_length, truncating="pre")
X_test_pad = keras.preprocessing.sequence.pad_sequences(
    X_test_tokens, maxlen=max_length, truncating="pre")

In [ ]:
print(f"Forme du corpus de documents : {X_train_pad.shape}")
print(f"Premier exemple : {X_train_pad[0]}")

## Modélisation

Construisez un modèle avec pour caractéristiques :
  - Une couche d'embeddings de taille 300
  - Une couche de GRU bidirectionnels (Bidirectional est un keras layer):
    - de taille `64`
    - une initialisation **des** matrices de poids orthogonales
    - un paramètre de dropout à `0.2` pour les parties forward et récurrentes
  - Une couche de réseau de neurones à activation `softmax` qui prend en entrée la dernière sortie de la couche [`GRU`](https://keras.io/api/layers/recurrent_layers/gru/)
  - Une fonction de perte basée sur l'**entropie croisée**
  - L'optimiseur `adam`
  - l'`accuracy` comme métrique d'évaluation

In [ ]:
# Votre code ici

### Solution

In [ ]:
EMBEDDING_DIM = 300

model = keras.models.Sequential()
model.add(keras.layers.Input(shape=max_length))
model.add(keras.layers.Embedding(vocab_size, EMBEDDING_DIM))
model.add(keras.layers.Bidirectional(
    keras.layers.GRU(64,
                     kernel_initializer="orthogonal",
                     recurrent_initializer="orthogonal",
                     dropout=0.2,
                     recurrent_dropout=0.2)))
model.add(keras.layers.Dense(9, activation="softmax"))
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])
model.summary()

## Apprentissage

Apprenez votre modèle sur `X_train_pad` :
- avec des batch de taille `1024`
- pendant `10` itérations
- en utilisant 30% de la base d'apprentissage pour validation


In [ ]:
# Votre code ici

### Solution

In [ ]:
model.fit(X_train_pad,
          y_train,
          batch_size=1024,
          epochs=10,
          validation_split=0.3)

## Évaluation

Évaluez les performances de votre modèle sur la base de test.

In [ ]:
# Votre code ici

### Solution

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test_pad, y_test)
print(f"Accuracy sur la base de test : {test_accuracy:.3f}")

## Premières conclusions

*Au vu des résultats, que peut-on dire de la qualité de cet apprentissage ? Justifiez.*

Votre réponse ici

### Solution

On observe un phénomène de surapprentissage. train $\approx$ 75% alors que validation et test $\approx$ 50%


## Utilisation d'embeddings de mots pré-appris

Afin d'obtenir de meilleurs résultats, vous allez utiliser les embeddings pré-appris GloVe, plutôt que d'apprendre des embeddings spécifiques comme précédemment.

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

In [ ]:
def glove_path(embedding_dim: int) -> pathlib.Path:
  if embedding_dim in {50, 100, 200, 300}:
    return pathlib.Path("glove.6B.{}d.txt".format(embedding_dim))
  else:
    raise ValueError("embedding_dim must be in {50, 100, 200, 300}")

## Réutilisation des Word Embeddings GloVe

La couche Embedding de Keras peut être initialisée avec une matrice de poids où la ligne i correspond à l'embedding du mot i.

Après un rapide coup d'oeil aux fichiers `glove.6B.300d.txt` :
- Créez un layer `embedding_layer` de type [`tensorflow.keras.layers.Embedding`](https://keras.io/api/layers/core_layers/embedding/) :
  - Initialisez-le avec les embeddings GloVe. 
  - Initialisez les mots de notre vocabulaire qui ne seraient pas dans GloVe avec le vecteur nul
  - Utilisez le flag approprié pour empêcher le changement de ces embeddings pendant l'apprentissage

In [ ]:
# Votre code ici

### Solution

In [ ]:
embedding_matrix = numpy.zeros((vocab_size, EMBEDDING_DIM))
found = 0
with glove_path(EMBEDDING_DIM).open() as fh:
  for line in fh:
    values = line.split(" ")
    word = values[0]
    if word in tokenizer_obj.word_index:
      found += 1
      coeffs = numpy.array(values[1:], dtype="float32")
      embedding_matrix[tokenizer_obj.word_index[word]] = coeffs

print(f"Utilisation de {found} embeddings pré-entraînés sur {vocab_size} mots "
      "dans le vocabulaire")

embedding_layer = keras.layers.Embedding(vocab_size,
                                         EMBEDDING_DIM,
                                         weights=[embedding_matrix],
                                         input_length=max_length,
                                         trainable=False)

## Modélisation, apprentissage et évaluation

- Créez un modèle identique à celui de la partie précédente, à ceci près que la couche d'embeddings est celle que l'on vient d'instancier à partir de GloVe
- Entraînez ce modèle avec les mêmes paramètres que dans la partie précédente
- Évaluez-le sur les données de test

NB : pour obtenir de meilleurs résultats qu'avec des SVMs, un modèle est proposé en solution.

In [ ]:
# Votre code ici

### Solution

In [ ]:
model = keras.models.Sequential()
model.add(embedding_layer)
model.add(keras.layers.Bidirectional(
    keras.layers.GRU(64,
                     kernel_initializer="orthogonal",
                     recurrent_initializer="orthogonal",
                     dropout=0.2,
                     recurrent_dropout=0.2)))
model.add(keras.layers.Dense(9, activation ="softmax"))

model.compile(loss="sparse_categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])
model.summary()

In [ ]:
model.fit(X_train_pad,
          y_train,
          batch_size=1024,
          epochs=20,
          validation_split=0.3)

In [ ]:
model.evaluate(X_test_pad, y_test)

## Solution meilleure que des SVM

In [ ]:
bidi_gru_params = dict(kernel_initializer="orthogonal",
                       recurrent_initializer="orthogonal",
                       dropout=0.2,
                       recurrent_dropout=0.2)

model = keras.models.Sequential()
model.add(embedding_layer)
model.add(keras.layers.Bidirectional(
    keras.layers.GRU(64, **bidi_gru_params, return_sequences=True)))
model.add(keras.layers.Bidirectional(keras.layers.GRU(64, **bidi_gru_params)))
model.add(keras.layers.Dense(9, activation="softmax"))

model.compile(loss="sparse_categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

model.summary()

In [ ]:
model.fit(X_train_pad,
          y_train,
          batch_size=1024,
          epochs=20,
          validation_split=0.3)

In [ ]:
model.evaluate(X_test_pad, y_test)

## Affichage de la matrice de confusion en Test

*Affichez la matrice de confusion (l'utilisation d'une heatmap est recommandée).*

In [ ]:
# Votre code ici

### Solution

In [ ]:
y_pred = model.predict_classes(X_test_pad)

In [ ]:
conf_mat = sklearn.metrics.confusion_matrix(y_pred, y_test, normalize="true")
seaborn.heatmap(conf_mat,
                vmin=0,
                vmax=1,
                cmap="rocket_r")